# Import

In [32]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
load_dotenv()
import os
OPENAI = os.getenv("OPENAI_KEY")
from openai import OpenAI
from tqdm import tqdm
import json
import re


### Cleaning

In [70]:
# Clean the unfiltered data
def clean_df(df: pd.DataFrame) -> pd.DataFrame:
    try:
        for elem in df["reviews"]:
            for review in elem:
                try:
                    del review["link"]
                    del review["source"]
                    del review["review_id"]
                    del review["user"]
                    del review["images"]
                except:
                    print("Key not found")
                    pass
    except Exception as e:
        print(e)
        df["reviews"] = "none"
        pass

    df.drop(["business_status", "icon", "icon_background_color", "icon_mask_base_uri", "opening_hours"], inplace=True, axis=1)    
    

    return df

In [71]:
df = pd.read_json("data-with-reviews.json")
# (df["reviews"] == "none").value_counts()
    # 50 rows have no reviews
df = df[df["reviews"] == "none"]
df = clean_df(df)

Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key not found
Key no

In [72]:
# # extract zip code
df['postal_code'] = df['formatted_address'].apply(lambda x: re.findall(r'\b\d{5}\b', x)[0] if re.findall(r'\b\d{5}\b', x) else None)
df.drop("formatted_address", inplace=True, axis=1)

# drop photos (all unique and non null)
df.drop(["photos"], inplace=True, axis=1)

# topics to boolean
df.loc[df["topics"] == "no topics", "topics"] = False
df.loc[df["topics"] != False, "topics"] = True

# extract lat and long
df['lat'] = df['geometry'].apply(lambda x: x['location']['lat'] if 'location' in x and 'lat' in x['location'] else None)
df['lng'] = df['geometry'].apply(lambda x: x['location']['lng'] if 'location' in x and 'lng' in x['location'] else None)
df.drop("geometry", inplace=True, axis=1)

# greater than 50 total reviews
df['greater_than_50'] = df['user_ratings_total'] > 50

# has certain keywords in "types"
df['is_dentist'] = df['types'].apply(lambda x: 'dentist' in x)
df['is_health'] = df['types'].apply(lambda x: 'health' in x)
df['is_establishment'] = df['types'].apply(lambda x: 'establishment' in x)
df['is_doctor'] = df['types'].apply(lambda x: 'doctor' in x)
df.drop(["types"], inplace=True, axis=1)

# drop other cols
df.drop(["place_id", "plus_code", "reference", "name", "reviews"], inplace=True, axis=1)

In [73]:
embedded_df = pd.read_csv("data_final.csv")

In [74]:
column_names = ['embedding_' + str(i) for i in range(1536)]

# Create DataFrame
embeddings_space = pd.DataFrame(columns=column_names)
# embeddings_space.head()
df = pd.concat([df, embeddings_space], axis=1)
df.fillna(0, inplace=True)

/var/folders/1t/0999bgxs5s17vns3pvsxhgr80000gn/T/ipykernel_25783/248425590.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(0, inplace=True)


In [75]:
result = pd.concat([embedded_df, df], ignore_index=True)
# result.head(-1)

In [76]:
test = result.dropna(axis="columns")

In [80]:
test.to_csv('data-with-null-embeddings.csv', index=False)

In [81]:
test.head()

,rating,user_ratings_total,topics,postal_code,lat,lng,greater_than_50,is_dentist,is_health,is_establishment,...,embedding_1526,embedding_1527,embedding_1528,embedding_1529,embedding_1530,embedding_1531,embedding_1532,embedding_1533,embedding_1534,embedding_1535
0,4.7,253.0,True,7230,36.861774,30.792039,True,True,True,True,...,0.005595,-0.016748,0.071510,-0.023414,-0.036926,-0.015911,-0.015298,0.007039,0.003957,-0.037707
1,4.8,135.0,True,7600,36.893972,30.691002,True,True,True,True,...,0.008192,-0.018630,0.062999,-0.033759,-0.053932,-0.012153,-0.007448,0.004616,0.000669,-0.030039
2,4.8,90.0,True,7050,36.886999,30.683048,True,True,True,True,...,0.007987,-0.007059,0.060596,-0.026944,-0.037144,-0.010489,-0.014042,0.004921,0.009987,-0.047537
3,4.9,216.0,True,7160,36.852699,30.751760,True,True,True,True,...,0.012997,-0.009758,0.062005,-0.037326,-0.040376,-0.008786,-0.010493,0.014760,0.004886,-0.039173
4,4.7,253.0,True,7230,36.861774,30.792039,True,True,True,True,...,0.005595,-0.016748,0.071510,-0.023414,-0.036926,-0.015911,-0.015298,0.007039,0.003957,-0.037707


## Fix the Fact that the Embeddings are in a Column Rather than all in a diff colum

In [73]:
# find out how long the longest embedding is
longest = df['embeddings'].apply(len).max()

# convert to json 
for elem in df["embeddings"]:
    elem = json.loads(elem)

# fill out empty with `None`
df['embeddings'] = df['embeddings'].apply(lambda x: x + [None] * (longest - len(x)))

# create `longest` amount of columns for the embeddings
# embeddings_df = pd.DataFrame(df['embeddings'].tolist(), columns=[f'embedding_{i}' for i in range(longest)])

TypeError: can only concatenate str (not "list") to str

In [59]:
# for i in range(longest):
#     embeddingList = list(df["embeddings"].iloc[i])

#     for j in range(len(embeddingList)):
#         df[f"embedding_{i}"].iloc[j] = embeddingList[j]

df.head()

,rating,user_ratings_total,topics,embeddings,postal_code,lat,lng,greater_than_50,is_dentist,is_health,...,embedding_34473,embedding_34474,embedding_34475,embedding_34476,embedding_34477,embedding_34478,embedding_34479,embedding_34480,embedding_34481,embedding_34482
0,4.7,253.0,True,"[-0.014175066724419594, 0.007809185888618231, ...",07230,36.8617744,30.7920385,True,True,True,...,None,None,None,None,None,None,None,None,None,None
1,4.8,135.0,True,"[-0.014537202194333076, 0.009266605600714684, ...",07600,36.8939721,30.6910022,True,True,True,...,None,None,None,None,None,None,None,None,None,None
2,4.8,90.0,True,"[-0.028139766305685043, 0.0001762386818882078,...",07050,36.886999,30.683048,True,True,True,...,None,None,None,None,None,None,None,None,None,None
3,4.9,216.0,True,"[-0.02719695121049881, 0.006841208320111036, 0...",07160,36.8526995,30.7517603,True,True,True,...,None,None,None,None,None,None,None,None,None,None
4,4.7,253.0,True,"[-0.014175066724419594, 0.007809185888618231, ...",07230,36.8617744,30.7920385,True,True,True,...,None,None,None,None,None,None,None,None,None,None


In [48]:
# df.head(-1)
len(df["embeddings"].iloc[43])
# embeddings_df = pd.DataFrame(df['embeddings'])
# combined_df = pd.concat([df, embeddings_df], axis=1)
# embeddings_df.head()

34408

# Findings
- \> 50 reviews looks to be negative
- "point_of_interest" does not matter